Load jupyter sql extension


In [1]:
#Load Jupter SQL extension
%reload_ext sql
%config SqlMagic.autocommit = False

In [2]:
import pandas as pd

In [3]:
#Load database
%sql sqlite:///../data/raw/frantamarit.db

'Connected: @../data/raw/frantamarit.db'

In [4]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable tables


In [5]:
%%sql master_join <<

SELECT
    rank_info.id,
    rank_info.name as fullName,
    rank_info.position,
    personal_info.lastName,
    personal_info.age,
    personal_info.gender,
    personal_info.country,
    personal_info.image,
    business_info.Source as source,
    business_info.worth,
    business_info.worthChange,
    business_info.realTimePosition
FROM 
    rank_info
LEFT JOIN 
    personal_info
ON
    rank_info.id = personal_info.id
LEFT JOIN
    business_info
ON 
    rank_info.id = business_info.id
ORDER BY
    rank_info.position DESC;

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable master_join


In [6]:
#Showing tables
df_master = master_join.DataFrame()

In [28]:
df_master

,id,fullName,position,lastName,age,gender,country,image,source,worth,worthChange,realTimePosition,sourceDetails
0,1690,OLGUN Zorlu,2208.0,zoRLU,54 years old,0,Turkey,https://specials-images.forbesimg.com/imageser...,Diversified,1.0,nan millions USD,1978.0,diversified
1,2599,zhuO Jun,2207.0,zhUO,52 years old,1,Hong Kong,https://specials-images.forbesimg.com/imageser...,Manufacturing,1.0,0.0 millions USD,1978.0,printed circuit boards
2,8536,ZHU XINGMING,2206.0,zHU,51 years old,-1,People's Republic of China,https://specials-images.forbesimg.com/imageser...,Manufacturing,1.0,0.0 millions USD,1856.0,electrical equipment
3,5887,ZHOU LIANGZHANG,2205.0,zhOU,55 years old,-1,None,https://specials-images.forbesimg.com/imageser...,Manufacturing,1.0,nan millions USD,2134.0,electrical equipment
4,9216,zhao XIAOQIANg,2204.0,zHAO,51 years old,0,China,https://specials-images.forbesimg.com/imageser...,Fashion & Retail,1.0,0.0 millions USD,2134.0,"fashion, entertainment"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,8911,supaluck umPUJH & Family,NaN,None,None,-1,None,None,NaN,NaN,None,NaN,NaN
2256,4098,thomaS Duff,NaN,None,None,-1,None,None,NaN,NaN,None,NaN,NaN
2257,6885,zakhar SMUSHkin,NaN,None,None,-1,None,None,NaN,NaN,None,NaN,NaN
2258,5401,zhang ZHONGNENG & family,NaN,None,None,-1,None,None,NaN,NaN,None,NaN,NaN


In [27]:
#To use afterwards
def lowercase_feature(df,col):
    return df[col].str.lower()

In [29]:
df_master['lastName'] = lowercase_feature(df_master,'lastName')

In [7]:
def extract_str(df,col,regex):
    return df[col].str.extract(regex)

In [8]:
df_master['sourceDetails'] = extract_str(df_master,'source',r"> (.*)")

In [9]:
df_master['source'] = extract_str(df_master,'source',r"(.*)  =")

In [25]:
df_master['worth'] = extract_str(df_master,'worth',r"(\d.*) ").astype('float64')

In [27]:
df_master['worthChange'].sort_values().unique()

array(['-0.001 millions USD', '-0.002 millions USD',
       '-0.013000000000000001 millions USD', '-0.043 millions USD',
       '-0.055999999999999994 millions USD', '-0.057 millions USD',
       '-0.068 millions USD', '-0.09 millions USD', '-0.109 millions USD',
       '-0.156 millions USD', '-0.16 millions USD', '-0.177 millions USD',
       '-0.179 millions USD', '-0.213 millions USD',
       '-0.243 millions USD', '-0.251 millions USD',
       '-0.254 millions USD', '-0.26 millions USD', '-0.268 millions USD',
       '-0.273 millions USD', '-0.299 millions USD',
       '-0.301 millions USD', '-0.392 millions USD',
       '-0.409 millions USD', '-0.6809999999999999 millions USD',
       '-0.7170000000000001 millions USD', '-0.805 millions USD',
       '-1.14 millions USD', '-1.171 millions USD',
       '-1.2570000000000001 millions USD', '-1.569 millions USD',
       '-17.012 millions USD', '-18.398 millions USD',
       '-18.864 millions USD', '-19.162 millions USD',
       '-2.001

In [15]:
df_master['gender'] = [1 if i == 'F' or  i == 'Female' else 0 if i == 'M' or i == 'Male' else -1 for i in df_master['gender']]

In [17]:
df_master[''].

count    2260.000000
mean       -0.190708
std         0.564089
min        -1.000000
25%        -1.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: gender, dtype: float64

1. Creating engine to connect to the database

In [5]:
#Not in use anymore from here until the end.

#from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import create_engine

2. Create engine and get data tables

In [7]:
#Creating engine
eng = create_engine('sqlite:///../data/raw/frantamarit.db')

3. Creating data frames for each table

In [8]:
#Creating function to ingest sql table and convert it to dataframe
def create_df_sql(x):
    return pd.read_sql_table(x, con=eng)

#Defining dataframes
df_business_info = create_df_sql('business_info')
df_personal_info = create_df_sql('personal_info')
df_rank_info = create_df_sql('rank_info')

In [9]:
def outer_merge(x, y, key):
    return pd.merge(x, y, on=key, how='outer')

In [10]:
df_partial_master = outer_merge(df_rank_info,df_personal_info,'id')

In [16]:
df_master =  outer_merge(df_partial_master, df_business_info, 'id')

In [17]:
df_master.sort_values('position',ascending=False)

,id,name,position,Unnamed: 0_x,lastName,age,Unnamed: 0_y,gender,country,image,Unnamed: 0,Source,worth,worthChange,realTimeWorth,realTimePosition
2259,1690,OLGUN Zorlu,2208.0,2259,zoRLU,54 years old,2259.0,Male,Turkey,https://specials-images.forbesimg.com/imageser...,2259.0,Diversified ==> diversified,1.0 BUSD,nan millions USD,NaN,1978.0
2258,2599,zhuO Jun,2207.0,2258,zhUO,52 years old,2258.0,F,Hong Kong,https://specials-images.forbesimg.com/imageser...,2258.0,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,NaN,1978.0
2257,8536,ZHU XINGMING,2206.0,2257,zHU,51 years old,2257.0,None,People's Republic of China,https://specials-images.forbesimg.com/imageser...,2257.0,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,NaN,1856.0
2256,5887,ZHOU LIANGZHANG,2205.0,2256,zhOU,55 years old,2256.0,None,None,https://specials-images.forbesimg.com/imageser...,2256.0,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,NaN,2134.0
2255,9216,zhao XIAOQIANg,2204.0,2255,zHAO,51 years old,2255.0,M,China,https://specials-images.forbesimg.com/imageser...,2255.0,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,NaN,2134.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,8911,supaluck umPUJH & Family,NaN,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,4098,thomaS Duff,NaN,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,6885,zakhar SMUSHkin,NaN,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,5401,zhang ZHONGNENG & family,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#Quick view of rank_info
df_personal_info.head()

,id,lastName,age,Unnamed: 0,gender,country,image
0,6528,beZOS,54 years old,52,M,None,https://specials-images.forbesimg.com/imageser...
1,5875,gaTES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...
2,8760,buFFEtt,87 years old,54,None,United States,https://specials-images.forbesimg.com/imageser...
3,4685,aRNAULT,69 years old,55,M,None,https://specials-images.forbesimg.com/imageser...
4,8584,zUCKERBERG,1985,56,M,United States,https://specials-images.forbesimg.com/imageser...


In [14]:
#Get unique values of column Unnamed: 0
df_rank_info['Unnamed: 0'].unique()

array([   0,    1,    2, ..., 2257, 2258, 2259])

In [15]:
#Looks like the column is redundant, so we will drop it
def drop_col_in(x,y):
    return x.drop(y, axis=1, inplace=True)

drop_col_in(df_rank_info, 'Unnamed: 0')

In [16]:
#Feature name is not written correctly, so let's harmonized it and lower case
#the name.
def lowercase_feature(x,y):
    x[y] = x[y].apply(lambda z: z.lower())
    return x[y]

lowercase_feature(df_rank_info, 'name')

0       anant asavabhokhin & family
1                    arash ferdowsi
2          banthoon lamsam & family
3            bill adderley & family
4                 boris zingarevich
                   ...             
2255                 zhao xiaoqiang
2256                zhou liangzhang
2257                   zhu xingming
2258                       zhuo jun
2259                    olgun zorlu
Name: name, Length: 2260, dtype: object

In [20]:
#We don't want to get rid of the NaN yet, so we'll fill them with 0s.
def fill_0(x):
    return x.fillna(0, inplace=True)

fill_0(df_rank_info)
df_rank_info

,id,name,position
0,5542,anant asavabhokhin & family,0.0
1,4183,arash ferdowsi,0.0
2,4761,banthoon lamsam & family,0.0
3,1449,bill adderley & family,0.0
4,1624,boris zingarevich,0.0
...,...,...,...
2255,9216,zhao xiaoqiang,2204.0
2256,5887,zhou liangzhang,2205.0
2257,8536,zhu xingming,2206.0
2258,2599,zhuo jun,2207.0


In [17]:
#get nulls

def get_null(x):
    return x.isnull().sum()

In [21]:
get_null(df_rank_info)

id          0
name        0
position    0
dtype: int64

In [32]:
len(df_rank_info)
#df_rank_info['Unnamed: 0'].le

2260

In [38]:
print(rank_info.notnull().sum())
print(personal_info.notnull().sum())

NameError: name 'rank_info' is not defined

In [ ]:
#Formula to drop column inplace
def drop_col_in(x,y):
    return x.drop(y, axis=1, inplace=True)

drop_col_in()

"""
rank_info.drop('Unnamed: 0', axis=1, inplace=True)
personal_info.drop('Unnamed: 0', axis=1, inplace=True)
business_info.drop('Unnamed: 0', axis=1, inplace=True)
"""

In [ ]:
personal_info.head()

In [ ]:
partial_info = pd.merge(rank_info, personal_info, on='id', how='inner')
master_info = pd.merge(partial_info, business_info, on='id', how='inner')
master_info.head()


In [ ]:
#Cleaning and lowercasing name and lastName features
master_info[['name','lastName']] = \
master_info[['name','lastName']].apply(lambda x: x.astype(str).str.lower())

In [ ]:
master_info.isnull().sum()

In [ ]:
#Renaming feature names
master_info.rename(index={e: e+1 for e in range(len(master_info))}, 
                columns={'name': 'full_name', 
                         'lastName': 'last_name',
                        'Source': 'source'}, 
                inplace=True)

In [ ]:
master_info.head()